In [ ]:
import itertools
import copy
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
from sklearn.datasets import fetch_openml
from tqdm import tqdm

plt.close('all')

mnist = fetch_openml('mnist_784')

def zeros_like_dict(d):
  return { k: zeros_like_dict(v) if isinstance(v, dict)
        else np.zeros_like(v)
        for k, v in d.items() }

def logistic(x):
  return 1.0 / (1 + np.exp(-x))

def grad_logistic(x):
  z = logistic(x)
  return z*(1-z)

def softmax(x):
  ## Max trick: subtract maximum value from each vector before computing exp to avoid
  ## overflow and numerical instabilities
  m = np.max(x, axis=0)
  return np.exp(x-m) / np.sum(np.exp(x-m), axis=0)

def compute_linear_transform(x, W, b):
  return W @ x + np.atleast_2d(b).T

def compute_hidden_layer(x, p):
  W = p['W']['hidden']
  b = p['b']['hidden']
  a = compute_linear_transform(x, W, b)
  return logistic(a)

def compute_output_layer(x, p):
  W = p['W']['output']
  b = p['b']['output']
  x_out = compute_linear_transform(x, W, b)
  return softmax(x_out)

def forward(x, p):
  # if x is a single vector of shape (N,) make it a proper column vector: shape (N, 1)
  if x.ndim < 2:
    x = np.atleast_2d(x).T
  h = compute_hidden_layer(x, p)
  return compute_output_layer(h, p)


def compute_loss(x, y, p):
  if x.ndim < 2:
    x = np.atleast_2d(x).T
  if y.ndim < 2:
    y = np.atleast_2d(y).T
  N = x.shape[1]
  prediction = forward(x, p)
  crossentropy = -np.sum(y * np.log(prediction))
  return crossentropy / N

def compute_grad_loss(x, y, p):
  if x.ndim < 2:
    x = np.atleast_2d(x).T
  if y.ndim < 2:
    y = np.atleast_2d(y).T
  grad = zeros_like_dict(p)
  N = x.shape[1]

  # Forward pass to get activations
  a_hidden = compute_linear_transform(x, p['W']['hidden'], p['b']['hidden'])
  x_hidden = logistic(a_hidden)
  prediction = compute_output_layer(x_hidden, p)

  # backprop through softmax
  error = prediction - y

  # Assume e,xh are the columns of error,x_hidden, ie the individual samples.
  # The sum over all samples of the outer product e*x_h is equivalent to error @ x_hidden.T
  grad['W']['output'] = 1/N * error @ x_hidden.T
  grad['b']['output'] = 1/N * np.sum(error, axis=1)

  # backprop to hidden layer
  hidden_layer_error = (p['W']['output'].T @ error)
  grad['W']['hidden'] = 1/N * (hidden_layer_error * grad_logistic(a_hidden)) @ x.T
  grad['b']['hidden'] = 1/N * np.sum(hidden_layer_error * grad_logistic(a_hidden), axis=1)

  crossentropy = -np.sum(y * np.log(prediction), axis=0)
  accuracy = np.sum(np.argmax(prediction, axis=0) == np.argmax(y, axis=0)) / x.shape[1] * 100
  return grad, np.sum(crossentropy) / N, accuracy

def update_params(p, grad, tau):
  for key1, sub_dict in p.items():
    for key2, param_array in sub_dict.items():
      p_block = param_array
      g_block = grad[key1][key2]
      p_block -= tau * g_block
  return p

def p2vec(p):
  pvec = np.array([])
  for key1, sub_dict in p.items():
    for key2, param_array in sub_dict.items():
      pvec = np.append(pvec, param_array.ravel())
  return pvec

def linesearch(x, y, p, grad, beta, sigma):
  ## backtracking linesearch with Armijo condition
  gvec = p2vec(grad)
  alpha = 1.0
  for m in range(100):    # max 100 backtracking steps, should never be reached in practice
    alpha = beta**m*2.0
    p_new = copy.deepcopy(p)
    p_new = update_params(p_new, grad, alpha)
    l_new = compute_loss(x, y, p_new)
    th = sigma*alpha*np.dot(gvec,-gvec)
    if l_new-l < th:
      return alpha

def setup_visualization():
  fig = plt.figure(figsize=(14, 8))
  gs_main = fig.add_gridspec(2, 2, width_ratios=[1, 2])
  ax_loss = fig.add_subplot(gs_main[0, 0]) # left side for loss & accuracy
  ax_acc = fig.add_subplot(gs_main[1, 0])
  gs_sub = gs_main[:, 1].subgridspec(4, 4, wspace=0.1, hspace=0.3) # right side for 4x4 grid
  ax_grid_dict = {}
  for i in range(1, 17): # 1 to 16 subplots in the 4x4 grid
      row = (i - 1) // 4
      col = (i - 1) % 4
      ax_grid_dict[i] = fig.add_subplot(gs_sub[row, col])
  return fig, (ax_loss, ax_acc, ax_grid_dict)

def update_visualization(fig, axes, vis_data):
  ax_loss, ax_acc, ax_grid_dict = axes
  loss, accuracy_train, accuracy_test, display_samples = vis_data
  ax_loss.clear()
  ax_loss.semilogy(loss)
  ax_loss.set_xlabel('Iteration')
  ax_loss.set_ylabel('Loss')
  ax_loss.set_title(f'loss: {loss[-1]:.4f}')
  ax_acc.clear()
  ax_acc.plot(accuracy_train, label='train')
  ax_acc.set_xlabel('Iteration')
  ax_acc.set_ylabel('Accuracy')
  if accuracy_test:
    ax_acc.plot(accuracy_test, label='test')
    ax_acc.set_title(f'accuracy train/test: {accuracy_train[-1]:.2f}/{accuracy_test[-1]:.2f}')
  else:
    ax_acc.set_title(f'accuracy train: {accuracy_train[-1]:.2f}')
  ax_acc.legend()
  for ax_key in ax_grid_dict:
      ax_grid_dict[ax_key].clear()

  y_predictions_for_display = forward(X[:, display_samples], p)
  counter = 0
  for m in range(2):
    for n in range(4):
      idx_input_plot = 4*2*m+n+1
      idx_output_plot = idx_input_plot + 4
      ax_grid_dict[idx_input_plot].imshow(X[:, display_samples[counter]].reshape(28, 28), cmap='gray')
      ax_grid_dict[idx_input_plot].set_title(f'Input image nr {display_samples[counter]}', fontsize=8)
      ax_grid_dict[idx_input_plot].axis('off')
      ax_grid_dict[idx_output_plot].stem(y_predictions_for_display[:, counter])
      color = 'g' if np.argmax(y_predictions_for_display[:, counter]) == np.argmax(Y[:, display_samples[counter]]) else 'r'
      ax_grid_dict[idx_output_plot].set_title(f'Output: {np.argmax(y_predictions_for_display[:, counter])}', fontsize=8, color=color)
      ax_grid_dict[idx_output_plot].set_xticks(range(10))
      ax_grid_dict[idx_output_plot].tick_params(axis='x', labelsize=6)
      ax_grid_dict[idx_output_plot].tick_params(axis='y', labelsize=6)
      counter += 1
  plt.tight_layout()
  clear_output(wait=True)     # reset output cell
  display(fig)


imgs = mnist.data.to_numpy().transpose()
X = imgs / 255.0
## one-hot encoding of labels
labels = np.array( [int(t) for t in mnist.target ] )
Y = np.zeros((10, labels.size))
Y[labels,range(labels.size)] = 1

# visualize training data
#fig = plt.figure(figsize=(14, 8))
#for m in range(4):
#  for n in range(6):
#    ax = fig.add_subplot(4, 6, 6*m+n+1)
#    ax.imshow(X[:,6*m+n].reshape(28, 28), cmap='gray')
#    ax.set_title(f'Label: {np.argmax(Y[:,6*m+n])}')
#plt.tight_layout()

train_size = 20000      # use that many samples for training
N_hidden = 6           # number of hidden units
d_in = X.shape[0]
d_out = Y.shape[0]

X_test = X[:,train_size:]
Y_test = Y[:,train_size:]
X = X[:,:train_size]
Y = Y[:,:train_size]


## initialize parameters (Glorot)
p = {'W': {}, 'b': {} }
limit = np.sqrt(6 / (d_in + N_hidden))
p['W']['hidden'] = np.random.uniform(-limit, limit, size=(N_hidden, d_in))
limit = np.sqrt(6 / (N_hidden + d_out))
p['W']['output'] = np.random.uniform(-limit, limit, size=(d_out, N_hidden))
p['b'] = {'hidden': np.zeros(N_hidden), 'output': np.zeros(d_out)}

## Optimization hyperparameters
num_iter = 100
tau = 0.2     # constant descent stepsize
sigma = 0.01    # parameter for linesaearch
beta = 0.5      # reduction factor linesearch

loss = []
accuracy_train = []
accuracy_test = []
display_samples = np.random.randint(0, X.shape[1], size=8)   # 8 random samples for display
fig, axes = setup_visualization()

## run gradient descent
with tqdm(total=num_iter) as pbar:
  for k in range(num_iter):
    grad, l, accuracy = compute_grad_loss(X, Y, p)
    loss.append(l)
    accuracy_train.append(accuracy)

    tau = linesearch(X, Y, p, grad, beta, sigma)   # compute optimal stepsize via lineasearch
    p = update_params(p, grad, tau)

    #prediction_test = forward(X_test, p)
    #acc_test = np.sum(np.argmax(prediction_test, axis=0) == np.argmax(Y_test, axis=0)) / X_test.shape[1] * 100
    #accuracy_test.append(acc_test)

    pbar.update(1)
    pbar.set_postfix({"loss": loss[-1]})
    if k % 10 == 0:
      vis_data = (loss, accuracy_train, accuracy_test, display_samples)
      update_visualization(fig, axes, vis_data)

print(f'Network with {p2vec(p).size} parameters, final loss {loss[-1]:.4f}')



In [ ]:
## verify gradient numerically

def grad_numeric(x, y, p):
  delta = 1e-4
  grad = zeros_like_dict(p)

  for key1, sub_dict in p.items():
    for key2, param_array in sub_dict.items():
      p_block = param_array   # references  the data in p
      g_block = grad[key1][key2]
      p_block_flat = p_block.reshape(-1)   # reshape() references the data
      g_block_flat = g_block.reshape(-1)

      ## estimate gradient numerically by perturbing each parameter
      ## and computing the value of the loss function
      for i in range(p_block_flat.size):
        original_value = p_block_flat[i]
        p_block_flat[i] = original_value + delta   # p_block_flat references the data, so this changes p
        loss_p = compute_loss(x, y, p)
        p_block_flat[i] = original_value - delta
        loss_m = compute_loss(x, y, p)

        # numerical gradient, central differences
        g_block_flat[i] = (loss_p - loss_m) / (2 * delta)
        p_block_flat[i] = original_value   # restore original value
  return grad

n_g = 5   # number of samples to compute gradient for
grad, _, _ = compute_grad_loss(X[:,:n_g], Y[:,:n_g], p)
grad_num = grad_numeric(X[:,:n_g], Y[:,:n_g], p)
for k in grad.keys():
  for l in grad[k].keys():
    #print(f'grad {k} {l}\n{grad[k][l]}')
    #print(f'grad_numeric {k} {l}\n{grad_num[k][l]}')
    print(f'diff {k} {l}: {np.sum(np.abs(grad[k][l] - grad_num[k][l]))}')
print(f'total diff: {np.sum(np.abs(p2vec(grad)-p2vec(grad_num)))}, number of elements in gradient: {p2vec(grad).size}')